In [1]:
from tensorflow import keras
import tensorflow as tf

import numpy as np

from tensorflow.keras.metrics import CategoricalAccuracy

import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from utils import load_config
from gpu.gpu import set_gpu_memory_growth
import argparse
from models.makemodels import make_vanilla_model,make_proto_model
from tensorflow.keras.preprocessing.image import load_img,img_to_array, ImageDataGenerator
import numpy as np
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.metrics import accuracy_score
from data_loader import get_loader

In [2]:
test_path = 'datasets/GTSRB/GTSRB_Test'
X = []
for f in os.listdir(test_path):
    file_path = os.path.join(test_path,f)
    X += [img_to_array(load_img(file_path,target_size=(64,64)))]
X = np.array(X)

In [3]:
df = pd.read_csv('datasets/GTSRB/GT-final_test.csv',sep=';')
y_true = to_categorical(np.array(df['ClassId']),num_classes=43)

In [59]:
datapath = 'datasets/GTSRB/all'
tr_datagen = ImageDataGenerator(rotation_range = 20,shear_range=0.2,height_shift_range=0.1,width_shift_range=0.1,horizontal_flip=True)
train_gen = tr_datagen.flow_from_directory(datapath,batch_size=128,class_mode='categorical',target_size=(64,64))

Found 39209 images belonging to 43 classes.


In [5]:
from models.makemodels import make_proto_model

In [84]:
clf = make_proto_model(backbone='densenet',input_shape=(64,64,3))

In [85]:
clf.load_weights('model_files/best_densenet2_gtsrb2tt100k_final.h5')

In [86]:
inp = keras.layers.Input((64,64,3))
enc = keras.Model(inputs=clf.get_layer('encoder').input,outputs=clf.get_layer('encoder').output)
x = enc(inp)
x = keras.layers.Dense(100,activation='relu')(x)
x = keras.layers.Dense(43,activation='softmax')(x)
clf_encoder = keras.Model(inputs=inp,outputs=x)

In [10]:
clf_encoder,encoder = make_vanilla_model(backbone='densenet',n_class=43,dim=48)

In [3]:
clf_encoder,encoder = make_vanilla_model(backbone='conv3b',n_class=43,dim=48)

In [87]:
for layer in clf_encoder.layers[:2]:
    layer.trainable = False

In [88]:
clf_encoder.summary()

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 model_19 (Functional)       (None, 300)               2173926   
                                                                 
 dense_19 (Dense)            (None, 100)               30100     
                                                                 
 dense_20 (Dense)            (None, 43)                4343      
                                                                 
Total params: 2,208,369
Trainable params: 34,443
Non-trainable params: 2,173,926
_________________________________________________________________


In [91]:
metric = CategoricalAccuracy()
train_loss_tracker = keras.metrics.Mean(name='loss')
optimizer = keras.optimizers.SGD(learning_rate=0.01,momentum=0.9)
loss_fun = keras.losses.CategoricalCrossentropy()
clf_encoder.compile(optimizer=optimizer,loss=loss_fun,metrics=metric)

In [31]:
indexes = np.arange(len(X))

In [37]:
len(X) // 128

98

In [51]:
for i in range(10)

12630

In [50]:
np.argmax(y_true[0:10],axis=-1)

array([16,  1, 38, 33, 11, 38, 18, 12, 25, 35], dtype=int64)

In [53]:
y_pred = clf_encoder(X[indexes[0:1263]])

In [55]:
def test():
    acc = 0
    for i in range(10):
        y_pred = clf_encoder(X[indexes[i*1263:(i+1)*1263]])
        acc += np.mean(keras.metrics.categorical_accuracy(y_true[i*1263:(i+1)*1263],y_pred))
    return acc / 10

In [77]:
clf_encoder.fit(train_gen,epochs=1)

307/307 [==============================] - 49s 159ms/step - loss: 0.1683 - categorical_accuracy: 0.9452


In [78]:
test()

0.9389548659324646

In [92]:
best_acc = 0
for _ in range(2):
    clf_encoder.fit(train_gen,epochs=1)
    #y_pred = clf_encoder(X)
    #acc = np.mean(keras.metrics.categorical_accuracy(y_true,y_pred))
    acc = test()
    if acc > best_acc:
        best_acc = acc
        print(f'best accuracy = {best_acc:.4}')

307/307 [==============================] - 51s 158ms/step - loss: 0.1508 - categorical_accuracy: 0.9472
best accuracy = 0.9241
307/307 [==============================] - 49s 159ms/step - loss: 0.1254 - categorical_accuracy: 0.9538
best accuracy = 0.9277


In [32]:
clf = make_proto_model(backbone='densenet',input_shape=(64,64,3))

In [35]:
clf.load_weights('model_files/best_densenet_belga2flick_mse_ft.h5')

In [36]:
inp = keras.layers.Input((64,64,3))
enc = keras.Model(inputs=clf.get_layer('encoder').input,outputs=clf.get_layer('encoder').output)

In [22]:
loader = get_loader('gtsrb') 
test_generator = loader.get_test_generator(batch=8,dim=64)

Loading all data


In [37]:
t = iter(test_generator)
[Xs,Xq],y = next(t)
Zs = enc(Xs)
Zq = enc(X)

In [25]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
knn = KNeighborsClassifier(n_neighbors=1,n_jobs=-1,metric='l2')

In [52]:
acc = 0
start = time()
y_train = to_categorical(np.arange(36),num_classes=36)
knn.fit(Zs,y_train);
pb = tf.keras.utils.Progbar(len(test_generator),verbose=1)
for i,z in enumerate(test_generator):
    [Xs,Xq],y_test = z
    Zq = enc(Xq)
    acc += knn.score(Zq,y_test)
    values=[('test acc',(acc / (i+1)))]
    pb.add(1,values=values)
end = time() - start


In [38]:
y_train = to_categorical(np.arange(43),num_classes=43)
knn.fit(Zs,y_train);

In [39]:
knn.score(Zq,y_true)

0.930720506730008